In [28]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

!pip install geopandas
!pip install geopy
from geopy.geocoders import Nominatim

In [12]:
url = 'https://www.citypopulation.de/en/trinidad/admin/'

In [10]:
read_table = pd.read_html(url, header=[0])
df = read_table[0]
df.head()

,Name,Status,PopulationCensus2000-05-15,PopulationCensus2011-01-09,Unnamed: 4
0,Arima,Borough,32278,33606,→
1,Arima Proper,Urban Community,9129,9082,→
2,Calvary Hill,Urban Community,1165,1330,→
3,Carib Homes,Urban Community,465,585,→
4,Malabar,Urban Community,8218,11130,→


In [20]:
df.drop(['PopulationCensus2000-05-15', 'Unnamed: 4'], axis=1, inplace=True)
df.rename(columns={'PopulationCensus2011-01-09':'Population'}, inplace=True)
df

,Name,Status,Population
0,Arima,Borough,33606
1,Arima Proper,Urban Community,9082
2,Calvary Hill,Urban Community,1330
3,Carib Homes,Urban Community,585
4,Malabar,Urban Community,11130
...,...,...,...
605,Valley View,Urban Community,1056
606,Valsayn,Urban Community,2552
607,Wallerfield,Rural Community,4218
608,Warren Village,Urban Community,2125


In [22]:
df.drop([609], inplace=True)
df

,Name,Status,Population
0,Arima,Borough,33606
1,Arima Proper,Urban Community,9082
2,Calvary Hill,Urban Community,1330
3,Carib Homes,Urban Community,585
4,Malabar,Urban Community,11130
...,...,...,...
604,Tunapuna,Urban Community,16701
605,Valley View,Urban Community,1056
606,Valsayn,Urban Community,2552
607,Wallerfield,Rural Community,4218


In [34]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent='myGeocoder')
location = []
#geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

#for i in range(5):
address = df.Name[i]
    
   # coords = None
    
    #while(coords is None):
g=locator.geocode('{}, Trinidad and Tobago'.format(address))
        
location.append(g)
print(location)

[Location(Arima, 300831, Trinidad and Tobago, (10.6372057, -61.2829762, 0.0))]


In [35]:
print(g)

Arima, 300831, Trinidad and Tobago


In [36]:
#for i in range(df.shape[0]):
df['Address'] = df['Name'].astype(str) + ', Trinidad and Tobago'
df.head()

,Name,Status,Population,Address
0,Arima,Borough,33606,"Arima, Trinidad and Tobago"
1,Arima Proper,Urban Community,9082,"Arima Proper, Trinidad and Tobago"
2,Calvary Hill,Urban Community,1330,"Calvary Hill, Trinidad and Tobago"
3,Carib Homes,Urban Community,585,"Carib Homes, Trinidad and Tobago"
4,Malabar,Urban Community,11130,"Malabar, Trinidad and Tobago"


In [37]:
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('White Land, Trinidad and Tobago',), **{}).
Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 1369, in getresponse
    response.begin()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 310, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 271, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/socket.py", line 589, in rea

In [38]:
df.head()

,Name,Status,Population,Address,location,point,latitude,longitude,altitude
0,Arima,Borough,33606,"Arima, Trinidad and Tobago","(Arima, 300831, Trinidad and Tobago, (10.63720...","(10.6372057, -61.2829762, 0.0)",10.637206,-61.282976,0.0
1,Arima Proper,Urban Community,9082,"Arima Proper, Trinidad and Tobago",None,None,NaN,NaN,NaN
2,Calvary Hill,Urban Community,1330,"Calvary Hill, Trinidad and Tobago","(Calvary Hill, Saint Ann's, Port of Spain, 190...","(10.7097744, -61.5090987, 0.0)",10.709774,-61.509099,0.0
3,Carib Homes,Urban Community,585,"Carib Homes, Trinidad and Tobago",None,None,NaN,NaN,NaN
4,Malabar,Urban Community,11130,"Malabar, Trinidad and Tobago","(Malabar, Arima, 300831, Trinidad and Tobago, ...","(10.6196194, -61.2774636, 0.0)",10.619619,-61.277464,0.0


In [39]:
df.shape

(609, 9)

In [40]:
from IPython.display import HTML
import base64 

def create_download_link( df, title = "Download CSV file", filename = "trinidad.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)